In [1]:
!pip install -U "autogen-agentchat" "autogen-ext[openai,azure]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.0/755.0 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33/33 [autogen-agentchat]autogen-agentchat]]nts]


In [ ]:
import pandas as pd
import sqlite3
import os
from dotenv import load_dotenv
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.teams import DiGraphBuilder, GraphFlow

In [2]:
load_dotenv()

True

#### Load data into SQLite db

In [2]:
DB_PATH = "bank_customers_data.db"

def load_data_to_sqlite(csv_path):
    df = pd.read_csv(csv_path)
    conn = sqlite3.connect(DB_PATH)
    df.to_sql("customers", conn, if_exists="replace", index=False)
    conn.close()
    return True

#### Tools

In [3]:
def run_sql_query(sql_query: str) -> str:
    conn = sqlite3.connect(DB_PATH)
    result = pd.read_sql_query(sql_query, conn)
    conn.close()
    return result.to_string(index=False)

In [ ]:
def fetch_table_columns() -> list:
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute(f"PRAGMA table_info(customers)")
    results = cursor.fetchall()
    columns = [column[1] for column in results]
    conn.close()
    return columns

#### Agent Definition

In [7]:
# Define a model client
model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    api_key=os.environ["OPENAI_API_KEY"],
)

In [ ]:
de_assistant = AssistantAgent(
        name="data_engineer",
        model_client=model_client,
        tools=[fetch_table_columns],  # Tool is callable with SQL query
        system_message="""
            You are a data engineer with who writes SQL queries on a SQLite database on table 'customers' containing customer banking data. Infer the column name from the database and table before creating the query.
            Your task is to interpret user questions and generate SQL queries.
        """,
        reflect_on_tool_use=True,
        model_client_stream=True,  # Enable streaming tokens from the model client.
    )

da_assistant = AssistantAgent(
        name="data_analyst",
        model_client=model_client,
        tools=[run_sql_query],  # Tool is callable with SQL query
        system_message="""
            You are a data analyst who reviews and executes SQL queries on a SQLite database on table 'customers' containing customer banking data.
            Your task is to execute the SQL queries provided by the data engineer and return the results. If the result is too big, summarize the results and return the summary.
        """,
        reflect_on_tool_use=True,
        model_client_stream=True,  # Enable streaming tokens from the model client.
    )

#### Main

In [ ]:
load_data_to_sqlite("./data/data.csv")

# Build the graph
builder = DiGraphBuilder()
builder.add_node(de_assistant).add_node(da_assistant)
builder.add_edge(de_assistant, da_assistant)

# Build and validate the graph
graph = builder.build()

# Create the flow
flow = GraphFlow([de_assistant, da_assistant], graph=graph)

# Run the agent and stream the messages to the console.
async def main() -> None:
    await Console(flow.run_stream(task="What is the average balance by gender, age, and job classification?"))
    # Close the connection to the model client.
    # await az_model_client.close()


# NOTE: if running this inside a Python script you'll need to use asyncio.run(main()).
await main()

#### Sample Output:
task="What is the average balance by gender, age, and job classification?"

---------- TextMessage (user) ----------
What is the average balance by gender, age, and job classification?
---------- ToolCallRequestEvent (data_engineer) ----------
[FunctionCall(id='call_9FbwC5N4flC68TukWcH0DQrs', arguments='{}', name='fetch_table_columns')]
---------- ToolCallExecutionEvent (data_engineer) ----------
[FunctionExecutionResult(content="['Customer ID', 'Name', 'Surname', 'Gender', 'Age', 'Region', 'Job Classification', 'Date Joined', 'Balance']", name='fetch_table_columns', call_id='call_9FbwC5N4flC68TukWcH0DQrs', is_error=False)]
---------- ModelClientStreamingChunkEvent (data_engineer) ----------
To find the average balance by gender, age, and job classification, here's the SQL query:

```sql
SELECT 
    Gender,
    Age,
    [Job Classification],
    AVG(Balance) AS Average_Balance
FROM 
    customers
GROUP BY 
    Gender, Age, [Job Classification];
```
---------- ToolCallRequestEvent (data_analyst) ----------
[FunctionCall(id='call_fYBvpSgDbMz6KARUj8jP17Hw', arguments='{"sql_query":"SELECT Gender, Age, [Job Classification], AVG(Balance) AS Average_Balance FROM customers GROUP BY Gender, Age, [Job Classification];"}', name='run_sql_query')]
---------- ToolCallExecutionEvent (data_analyst) ----------
[FunctionExecutionResult(content='Gender  Age Job Classification  Average_Balance\nFemale   17              Other     65798.400000\nFemale   18              Other     51171.290000\nFemale   18       White Collar     45833.310000\nFemale   19              Other      9535.555000\nFemale   19       White Collar     30972.980000\nFemale   20              Other     10453.930000\nFemale   20       White Collar      1488.040000\nFemale   21        Blue Collar     60338.070000\nFemale   21              Other     13487.900000\nFemale   21       White Collar     25257.990000\nFemale   22              Other     68922.975000\nFemale   22       White Collar     52006.435000\nFemale   23        Blue Collar     38415.813333\nFemale   23              Other     33231.110000\nFemale   23       White Collar     36793.577500\nFemale   24        Blue Collar     10816.330000\nFemale   24              Other     55760.010000\nFemale   24       White Collar     47314.206667\nFemale   25        Blue Collar     47651.750000\nFemale   25              Other     36950.000000\nFemale   25       White Collar     41616.595000\nFemale   26        Blue Collar     39944.640000\nFemale   26              Other     32940.490000\nFemale   26       White Collar     38309.007273\nFemale   27              Other     17121.962500\nFemale   27       White Collar     44208.080000\nFemale   28              Other     40905.616667\nFemale   28       White Collar     32113.793333\nFemale   29        Blue Collar     18601.690000\nFemale   29              Other     28543.510000\nFemale   29       White Collar     23537.557000\nFemale   30        Blue Collar     53395.210000\nFemale   30              Other     24881.780000\nFemale   30       White Collar     28822.326250\nFemale   31        Blue Collar     35364.136667\nFemale   31              Other     45644.170000\nFemale   31       White Collar     53299.250769\nFemale   32        Blue Collar     23059.560000\nFemale   32              Other     51650.671667\nFemale   32       White Collar     48677.242000\nFemale   33        Blue Collar     48473.120000\nFemale   33              Other     53092.360000\nFemale   33       White Collar     32843.090000\nFemale   34        Blue Collar     43526.100000\nFemale   34              Other     84913.522000\nFemale   34       White Collar     34261.667857\nFemale   35        Blue Collar     64233.870000\nFemale   35              Other     30452.315000\nFemale   35       White Collar     26560.482500\nFemale   36        Blue Collar     42180.646000\nFemale   36              Other     37364.260000\nFemale   36       White Collar     42723.190000\nFemale   37        Blue Collar      9178.750000\nFemale   37              Other     43028.860000\nFemale   37       White Collar     26743.688571\nFemale   38        Blue Collar     41472.085000\nFemale   38              Other     26489.804444\nFemale   38       White Collar     36727.857000\nFemale   39              Other     23852.485000\nFemale   39       White Collar     50652.051250\nFemale   40        Blue Collar     51864.200000\nFemale   40              Other     35058.945000\nFemale   40       White Collar     32030.137273\nFemale   41        Blue Collar     18907.190000\nFemale   41              Other     54723.720000\nFemale   41       White Collar     21308.726667\nFemale   42        Blue Collar     25968.512500\nFemale   42              Other     37523.026000\nFemale   42       White Collar     32012.122857\nFemale   43        Blue Collar     19713.305000\nFemale   43              Other     69834.785000\nFemale   43       White Collar     34158.440000\nFemale   44              Other     28448.606667\nFemale   44       White Collar     15531.086667\nFemale   45        Blue Collar      2575.630000\nFemale   45              Other     36819.570000\nFemale   45       White Collar     47336.331429\nFemale   46        Blue Collar     18294.960000\nFemale   46       White Collar     31509.067500\nFemale   47        Blue Collar      9098.320000\nFemale   47              Other     38564.100000\nFemale   47       White Collar     42680.197500\nFemale   48        Blue Collar     46475.720000\nFemale   48              Other     35162.263333\nFemale   48       White Collar     26493.260000\nFemale   49              Other      9119.520000\nFemale   49       White Collar     20738.670000\nFemale   50              Other     16326.206667\nFemale   50       White Collar     57622.790000\nFemale   51        Blue Collar     40918.080000\nFemale   51              Other     47445.605000\nFemale   51       White Collar     19453.065000\nFemale   52        Blue Collar     76378.230000\nFemale   52              Other     36149.620000\nFemale   53              Other     42526.845000\nFemale   53       White Collar     35865.540000\nFemale   54              Other     31846.935000\nFemale   55              Other      9885.690000\nFemale   55       White Collar     35939.040000\nFemale   56              Other     26675.373333\nFemale   57              Other     22686.000000\nFemale   57       White Collar     45944.180000\nFemale   58              Other     43520.970000\nFemale   59              Other     38383.646667\nFemale   59       White Collar     44129.740000\nFemale   61              Other      8367.590000\nFemale   63              Other     18517.010000\n  Male   18       White Collar     15232.910000\n  Male   19        Blue Collar     94786.370000\n  Male   19       White Collar     70407.910000\n  Male   20        Blue Collar      9578.650000\n  Male   20       White Collar      7369.925000\n  Male   21       White Collar     70416.545000\n  Male   22        Blue Collar       837.330000\n  Male   22       White Collar      7202.520000\n  Male   23        Blue Collar     57251.455000\n  Male   23       White Collar     12665.996667\n  Male   24        Blue Collar     36839.126667\n  Male   24              Other     36256.200000\n  Male   24       White Collar     47306.097500\n  Male   25              Other      4203.533333\n  Male   25       White Collar     31660.860000\n  Male   26        Blue Collar     28210.780000\n  Male   26              Other     13988.695000\n  Male   26       White Collar      8031.898000\n  Male   27        Blue Collar     51412.600000\n  Male   27              Other     15502.375000\n  Male   27       White Collar     38464.014444\n  Male   28        Blue Collar     25473.533333\n  Male   28              Other     18064.460000\n  Male   28       White Collar    107024.390000\n  Male   29        Blue Collar     46661.913333\n  Male   29              Other    106172.130000\n  Male   29       White Collar     41177.936250\n  Male   30        Blue Collar     18068.060000\n  Male   30       White Collar     27516.072222\n  Male   31        Blue Collar     37404.913333\n  Male   31              Other     13313.475000\n  Male   31       White Collar     58752.402000\n  Male   32        Blue Collar     17749.950000\n  Male   32              Other     39096.812500\n  Male   32       White Collar     41228.466923\n  Male   33        Blue Collar     53376.636000\n  Male   33              Other     36680.136667\n  Male   33       White Collar     38437.611250\n  Male   34        Blue Collar     48564.935000\n  Male   34              Other     39892.760000\n  Male   34       White Collar     27422.205000\n  Male   35        Blue Collar     17550.043333\n  Male   35              Other     37464.240000\n  Male   35       White Collar     41728.760909\n  Male   36        Blue Collar     42862.854000\n  Male   36              Other       401.130000\n  Male   36       White Collar     36088.972000\n  Male   37        Blue Collar     45255.350000\n  Male   37              Other     44234.520000\n  Male   37       White Collar     45925.041818\n  Male   38        Blue Collar     40764.078333\n  Male   38       White Collar     39769.489000\n  Male   39        Blue Collar     30364.123333\n  Male   39              Other     56514.660000\n  Male   39       White Collar     40165.146667\n  Male   40        Blue Collar     27094.806000\n  Male   40              Other     14407.495000\n  Male   40       White Collar     34981.104000\n  Male   41        Blue Collar     54448.205000\n  Male   41              Other      4846.570000\n  Male   41       White Collar     60614.952727\n  Male   42        Blue Collar     61008.830000\n  Male   42              Other     37943.435000\n  Male   42       White Collar     31708.356667\n  Male   43        Blue Collar     50789.726667\n  Male   43              Other     26952.848000\n  Male   43       White Collar     27790.510000\n  Male   44        Blue Collar     42750.358333\n  Male   44              Other     15364.023333\n  Male   44       White Collar     43555.370000\n  Male   45        Blue Collar     41668.941667\n  Male   45              Other     34355.481538\n  Male   45       White Collar     12276.505000\n  Male   46        Blue Collar     34306.523333\n  Male   46              Other     48791.460000\n  Male   46       White Collar     63015.400000\n  Male   47        Blue Collar     28374.645714\n  Male   47              Other     26924.166667\n  Male   47       White Collar     32715.802500\n  Male   48        Blue Collar     21136.713333\n  Male   48              Other     41166.146000\n  Male   48       White Collar     47226.280000\n  Male   49        Blue Collar     74855.292857\n  Male   49       White Collar     68252.300000\n  Male   50        Blue Collar     50457.317500\n  Male   50              Other     48457.180000\n  Male   50       White Collar     43234.210000\n  Male   51        Blue Collar     49589.766364\n  Male   51              Other     32382.550000\n  Male   51       White Collar     59878.793333\n  Male   52        Blue Collar     35834.563333\n  Male   52              Other     34702.108000\n  Male   52       White Collar     68578.375000\n  Male   53        Blue Collar     39020.856000\n  Male   53              Other     75635.823333\n  Male   53       White Collar     22354.530000\n  Male   54        Blue Collar     44106.333333\n  Male   54              Other      5478.895000\n  Male   54       White Collar     52802.850000\n  Male   55        Blue Collar     35344.763333\n  Male   55              Other     36325.350000\n  Male   56        Blue Collar     31073.536667\n  Male   56              Other     70950.090000\n  Male   57        Blue Collar     39188.843333\n  Male   58        Blue Collar     29755.800000\n  Male   58              Other     70390.730000\n  Male   59        Blue Collar     27446.900000\n  Male   59              Other    114456.230000\n  Male   60        Blue Collar     13764.720000\n  Male   60              Other     49081.500000\n  Male   60       White Collar      1860.460000\n  Male   61        Blue Collar     31880.076667\n  Male   62        Blue Collar      6273.930000\n  Male   62              Other     53342.100000\n  Male   63        Blue Collar     26474.230000\n  Male   63              Other     48244.175000\n  Male   64        Blue Collar      9842.700000', name='run_sql_query', call_id='call_fYBvpSgDbMz6KARUj8jP17Hw', is_error=False)]
---------- ModelClientStreamingChunkEvent (data_analyst) ----------
The table contains detailed average balances segmented by gender, age, and job classification. To summarize:

1. **Gender Specifics**:
   - Both male and female average balances vary widely depending on combinations of age and job classification.

2. **Age Trends**:
   - Across genders, average balances generally fluctuate as age progresses, often influenced by job classifications like "White Collar," "Blue Collar," and "Other."

3. **Job Classification**:
   - "White Collar" jobs often have a higher average balance compared to "Blue Collar" or "Other," but exceptions exist depending on age and gender.

Let me know if you'd like me to filter or further analyze the data for specific insights!
---------- StopMessage (DiGraphStopAgent) ----------
Digraph execution is complete